In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.learn.manager import ClassificationManager, cm
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.modes import AvirisDataManager
from typing import List, Union, Tuple, Optional, Dict, Callable

2023-01-27 13:16:00.478075: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [2]:
dm: DataManager = DataManager.initialize( "AGB", 'neon' )


dm.modal.cache_dir = "/explore/nobackup/projects/ilab/cache"
dm.modal.data_dir  = "/explore/nobackup/projects/ilab/data"

block_size = 150
method = "aec" # "vae"
model_dims = 32
year= 2015
version = "beta_pmm"
roi = "541567.6_4136443.0_542567.6_4137443.0"

dm.modal.ext =  "_img"
dm.proc_type = "cpu"
dm.modal.images_glob = f"AGB/test/{version}/MLBS_{year}_{roi}/MLBS_{year}_Reflectance_reflectance_warp.tif"
TileManager.block_size = block_size
TileManager.reprocess = False
dm.modal.model_dims = model_dims
dm.modal.reduce_method = method
dm.modal.reduce_nepoch = 2
dm.modal.reduce_focus_nepoch = 0
dm.modal.reduce_niter = 12
dm.modal.reduce_focus_ratio = 10.0
dm.modal.reduce_dropout = 0.0
dm.modal.reduce_learning_rate = 1e-4
dm.modal.refresh_model = True
dm.modal.reduce_nblocks = 1000
dm.modal.reduce_nimages = 100
dm.modal.modelkey = f"b{block_size}.{version}.{year}.{roi}"

Opening log file:  '/home/tpmaxwel/.spectraclass/logging/aviris/img_mgr.log'
Using config file: '/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/defaults/config.py'
Using config file: '/home/tpmaxwel/.spectraclass/config/aviris/img_mgr.py'


/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3194: UserWarning: Config option `use_model_data` not recognized by `DataManager`.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/panfs/ccds02/home/tpmaxwel/JupyterLinks/spectraclass-dev/notebooks/spectraclass/model/base.py:26: UserWarning: Config option `reduce_target_block` not recognized by `AvirisDataManager`.  Did you mean one of: `reduce_method, reduce_nblocks, reduce_nepoch`?
  inst = cls(*args, **kwargs)


Here we load the project data and define a set of class names and associated colors to be used in the classification process.

In [3]:
dm.loadCurrentProject()
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

lm().setLabels( classes )

Completed Reading raster file /css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data//ang20190801t150124rfl/ang20190801t150124_rfl_v2v2/ang20190801t150124_corr_v2v2_img, dims = ('band', 'y', 'x'), shape = (425, 14400, 724)
#Tile[0]-> Read Data: shape = (425, 14400, 724), dims=('band', 'y', 'x')


Add Custom CNN

In [4]:
from spectraclass.learn.mlp import MLP
from spectraclass.learn.cnn import CNN2D, SpectralCNN, CNN3D

cm().nepochs = 20
cm().mid = "mlp"

mlp = MLP( 'mlp', layers = [32, 16] )
cm().addNetwork(mlp)

cnn = CNN2D( 'cnn2d', cnn_layers =  [(8,3,1)], dense_layers= [32, 16] )
cm().addNetwork(cnn)

cnn1D = SpectralCNN( 'cnn1d', cnn_layers = [(8,5,3),(8,5,3)], dense_layers= [32, 16] )
cm().addNetwork(cnn1D)

/tmp/ipykernel_2789978/3003413587.py:4: UserWarning: Config option `nfeatures` not recognized by `ClassificationManager`.
  cm().nepochs = 20
/tmp/ipykernel_2789978/3003413587.py:4: UserWarning: Config option `cnn_layers` not recognized by `ClassificationManager`.
  cm().nepochs = 20
/tmp/ipykernel_2789978/3003413587.py:4: UserWarning: Config option `dense_layers` not recognized by `ClassificationManager`.
  cm().nepochs = 20
2023-01-27 13:16:33.261316: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 363)]             0         
                                                                 
 dense_3 (Dense)             (None, 182)               66248     
                                                                 
 dense_4 (Dense)             (None, 91)                16653     
                                                                 
 dense_5 (Dense)             (None, 46)                4232      
                                                                 
 dense_7 (Dense)             (None, 64)                3008      
                                                                 
 dense_8 (Dense)             (None, 128)               8320      
                                                                 
 dense_9 (Dense)             (None, 256)               33024 

Here we start up the Spectraclass GUI.

In [5]:
controller: SpectraclassController = app()
controller.gui()

Initializing GUI using controller <class 'spectraclass.gui.spatial.application.Spectraclass'>
GUI Init complete
